In [164]:
!pip uninstall -y torch torchvision torchaudio torch-geometric
!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cpu
!pip install torch-geometric==2.5.2


Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: torchvision 0.17.2
Uninstalling torchvision-0.17.2:
  Successfully uninstalled torchvision-0.17.2
Found existing installation: torchaudio 2.2.2
Uninstalling torchaudio-2.2.2:
  Successfully uninstalled torchaudio-2.2.2
Found existing installation: torch_geometric 2.5.2
Uninstalling torch_geometric-2.5.2:
  Successfully uninstalled torch_geometric-2.5.2
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.2.2-cp311-none-macosx_11_0_arm64.whl (59.3 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.17.2-cp311-cp311-macosx_11_0_arm64.whl (1.6 MB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.2.2-cp311-cp311-macosx_11_0_arm64.whl (1.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [torchaudio]3 [torchaudio]]
  Using cached torch_geometric-2.5.2-py3-non

In [165]:
import os
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import HeteroData

In [166]:
print("Torch:", torch.__version__)
print("PyG:", torch_geometric.__version__)
print("CUDA available:", torch.cuda.is_available())

Torch: 2.2.2
PyG: 2.5.2
CUDA available: False


In [167]:
# PyTorch (CPU 전용)
!pip install torch
# PyTorch Geometric (PyG CPU 빌드)
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.1.0+cpu.html

# 과학계산/데이터 처리 패키지
!pip install numpy pandas scipy

# (옵션) 데이터 구조
!pip install networkx


Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 652.1/652.1 kB 2.2 MB/s  0:00:00eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.4/975.4 kB 2.6 MB/s  0:00:0036m-:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.2/573.2 kB 1.9 MB/s  0:00:0036m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [torch-cluster]


# Troubleshooting

In [38]:
for name, M in mps.items():
    print(f"[{name}] shape={M.shape}, nnz={M.nnz}")

    pairs = csr_to_pairs(M, max_pos_per_node=20, seed=42)
    print(f"  pairs.shape={pairs.shape}")
    if pairs.shape[0] > 0:
        print("  sample:", pairs[:5])


[PAP] shape=(5000, 5000), nnz=0
  pairs.shape=(2, 0)
  sample: []
[PCP] shape=(5000, 5000), nnz=0
  pairs.shape=(2, 0)
  sample: []


In [40]:
print("edge_wrote:", data['author','wrote','paper'].edge_index.shape)
print("edge_has_topic:", data['paper','has_topic','concept'].edge_index.shape)

print("A_AP shape:", A_AP.shape, "nnz:", A_AP.nnz)
print("A_PC shape:", A_PC.shape, "nnz:", A_PC.nnz)


edge_wrote: torch.Size([2, 40758])
edge_has_topic: torch.Size([2, 82238])
A_AP shape: (33607, 5000) nnz: 1
A_PC shape: (5000, 6901) nnz: 82238


저자–논문 adjacency에 단 1개 엣지만 살아남음 (4만 건 중 1개만).

In [44]:
print("Author ID max:", df_wrote['src'].max(), "expected <", A)
print("Paper ID max:", df_wrote['dst'].max(), "expected <", P)
print("Author ID min:", df_wrote['src'].min())
print("Paper ID min:", df_wrote['dst'].min())


Author ID max: nan expected < 33607
Paper ID max: nan expected < 5000
Author ID min: nan
Paper ID min: nan


In [48]:
df_wrote = pd.read_csv("edge_wrote.csv")
print(df_wrote.head())
print(df_wrote.dtypes)
print(df_wrote.isna().sum())


     src  dst  author_order
0   8136    0             1
1  17961    0             2
2   1297    0             3
3  13091    0             4
4   5067    1             1
src             int64
dst             int64
author_order    int64
dtype: object
src             0
dst             0
author_order    0
dtype: int64


In [50]:
print("edge_ap shape:", data["author","wrote","paper"].edge_index.shape)
print("max author id:", data["author","wrote","paper"].edge_index[0].max().item(), "expected <", A)
print("max paper id:", data["author","wrote","paper"].edge_index[1].max().item(), "expected <", P)


edge_ap shape: torch.Size([2, 40758])
max author id: 0 expected < 33607
max paper id: 0 expected < 5000


In [54]:
df_wrote = pd.read_csv("edge_wrote.csv", dtype={"src": int, "dst": int})
print(df_wrote.dtypes)  # 확인
print(df_wrote.head())  # 다시 확인


src             int64
dst             int64
author_order    int64
dtype: object
     src  dst  author_order
0   8136    0             1
1  17961    0             2
2   1297    0             3
3  13091    0             4
4   5067    1             1


In [56]:
edge_ap = torch.tensor(
    df_wrote[['src','dst']].to_numpy(dtype=np.int64).T,
    dtype=torch.long
)
data['author','wrote','paper'].edge_index = edge_ap


In [58]:
print("edge_ap shape:", edge_ap.shape)
print("max author id:", edge_ap[0].max().item(), "expected <", A)
print("max paper id:", edge_ap[1].max().item(), "expected <", P)


edge_ap shape: torch.Size([2, 40758])
max author id: 33606 expected < 33607
max paper id: 4999 expected < 5000


In [60]:
print("edge_pc shape:", edge_pc.shape)
print("max paper id:", edge_pc[0].max().item(), "expected <", P)
print("max concept id:", edge_pc[1].max().item(), "expected <", C)


edge_pc shape: torch.Size([2, 82238])
max paper id: 0 expected < 5000
max concept id: 0 expected < 6901


In [62]:
df_wrote = pd.read_csv("edge_has_topic.csv", dtype={"src": int, "dst": int})
print(df_wrote.dtypes)  # 확인
print(df_wrote.head())  # 다시 확인


src                int64
dst                int64
concept_level      int64
concept_score    float64
dtype: object
   src   dst  concept_level  concept_score
0    0  3526              2       0.812645
1    0  1937              2       0.563623
2    0   552              1       0.410001
3    0  5006              0       0.368186
4    0   539              0       0.313520


In [67]:
import pandas as pd

paths = {
    "wrote": "edge_wrote.csv",
    "has_topic": "edge_has_topic.csv",
    "cites": "edge_cites.csv",
    "worked_with": "edge_worked_with.csv"
}

for name, path in paths.items():
    print(f"\n=== {name} ({path}) ===")
    df = pd.read_csv(path)

    # 컬럼명 및 타입
    print("컬럼:", df.columns.tolist())
    print(df.dtypes)

    # 행 개수, 고유값 개수
    print("행 수:", len(df))
    for col in df.columns:
        print(f"  - {col}: unique={df[col].nunique()}  null={df[col].isna().sum()}")

    # 샘플 5개
    print(df.head(5))



=== wrote (edge_wrote.csv) ===
컬럼: ['src', 'dst', 'author_order']
src             int64
dst             int64
author_order    int64
dtype: object
행 수: 40758
  - src: unique=33607  null=0
  - dst: unique=5000  null=0
  - author_order: unique=100  null=0
     src  dst  author_order
0   8136    0             1
1  17961    0             2
2   1297    0             3
3  13091    0             4
4   5067    1             1

=== has_topic (edge_has_topic.csv) ===
컬럼: ['src', 'dst', 'concept_level', 'concept_score']
src                int64
dst                int64
concept_level      int64
concept_score    float64
dtype: object
행 수: 82238
  - src: unique=5000  null=0
  - dst: unique=6901  null=0
  - concept_level: unique=6  null=0
  - concept_score: unique=64396  null=0
   src   dst  concept_level  concept_score
0    0  3526              2       0.812645
1    0  1937              2       0.563623
2    0   552              1       0.410001
3    0  5006              0       0.368186
4    0   53

In [69]:
df["author_id"].astype(str).map(lambda x: f"'{x}'").head(10)


KeyError: 'author_id'

# pkl 만들기

In [17]:
import os
import pandas as pd
import numpy as np
import torch

# ==========================
# 1. Edge CSV 점검 함수
# ==========================
def check_edge_csv(path, name, nrows=5):
    print(f"\n=== {name} ({path}) ===")
    if not os.path.exists(path):
        print("❌ 파일 없음")
        return
    df = pd.read_csv(path)
    print("컬럼:", df.columns.tolist())
    print(df.dtypes)
    print("행 수:", len(df))
    for col in df.columns:
        print(f"  - {col}: unique={df[col].nunique()} null={df[col].isna().sum()} "
              f"min={df[col].min()} max={df[col].max()}")
    print("샘플 rows:")
    print(df.head(nrows))
    return df

# ==========================
# 2. Feature 텐서 점검 함수
# ==========================
def check_feats(feat_dict):
    for k, v in feat_dict.items():
        print(f"\n=== {k} feats ===")
        if isinstance(v, torch.Tensor):
            print("torch.Tensor", v.shape, v.dtype)
        elif isinstance(v, np.ndarray):
            print("numpy.ndarray", v.shape, v.dtype)
        elif isinstance(v, str) and os.path.exists(v):
            # 파일 경로라면 로드해서 확인
            if v.endswith(".pt") or v.endswith(".pkl"):
                t = torch.load(v)
                print(f"loaded torch {type(t)} shape={getattr(t, 'shape', None)}")
            elif v.endswith(".npy"):
                a = np.load(v)
                print("loaded numpy", a.shape, a.dtype)
            elif v.endswith(".csv"):
                df = pd.read_csv(v)
                print("loaded csv", df.shape, "cols:", df.columns.tolist())
                print(df.head(3))
        else:
            print("❓ 알 수 없는 타입:", type(v))

# ==========================
# 실행 예시
# ==========================
# 1) Edge CSV 확인
df_ap = check_edge_csv("edge_wrote.csv", "wrote")
df_pc = check_edge_csv("edge_has_topic.csv", "has_topic")
df_pp = check_edge_csv("edge_cites.csv", "cites")
df_aa = check_edge_csv("edge_worked_with.csv", "worked_with")

# 2) 노드 임베딩 확인 (여기에 실제 경로/객체 넣어주세요)
# 예시: feats = {"paper": "paper_feats.npy", "author": "author_feats.npy", "concept": "concept_feats.npy"}
# 또는 이미 torch.Tensor라면 그대로 넣어도 됩니다
feats = {
    "paper":   "paper_feats.npy",
    "author":  "author_feats.npy",
    "concept": "concept_feats.npy",
}
check_feats(feats)



=== wrote (edge_wrote.csv) ===
컬럼: ['src', 'dst', 'author_order']
src             int64
dst             int64
author_order    int64
dtype: object
행 수: 40758
  - src: unique=33607 null=0 min=0 max=33606
  - dst: unique=5000 null=0 min=0 max=4999
  - author_order: unique=100 null=0 min=1 max=100
샘플 rows:
     src  dst  author_order
0   8136    0             1
1  17961    0             2
2   1297    0             3
3  13091    0             4
4   5067    1             1

=== has_topic (edge_has_topic.csv) ===
컬럼: ['src', 'dst', 'concept_level', 'concept_score']
src                int64
dst                int64
concept_level      int64
concept_score    float64
dtype: object
행 수: 82238
  - src: unique=5000 null=0 min=0 max=4999
  - dst: unique=6901 null=0 min=0 max=6900
  - concept_level: unique=6 null=0 min=0 max=5
  - concept_score: unique=64396 null=0 min=0.0 max=0.9991346
샘플 rows:
   src   dst  concept_level  concept_score
0    0  3526              2       0.812645
1    0  1937        

In [19]:
import pandas as pd
import os

# 노드 임베딩 파일 경로
node_files = {
    "paper":   "cs_nodes_paper_emb.csv",   # 오타 아닌지 확인! (papar → paper?)
    "author":  "cs_nodes_author_emb.csv",
    "concept": "cs_nodes_concept_emb2.csv",
}

def check_node_csv(path, name, nrows=5):
    print(f"\n=== {name} ({path}) ===")
    if not os.path.exists(path):
        print("❌ 파일 없음")
        return
    df = pd.read_csv(path)
    print("컬럼 수:", df.shape[1], "행 수:", df.shape[0])
    print("컬럼명:", df.columns.tolist()[:10], "..." if df.shape[1] > 10 else "")
    print(df.dtypes.head(10))
    print("샘플 rows:")
    print(df.head(nrows))
    return df

dfs = {}
for k, path in node_files.items():
    dfs[k] = check_node_csv(path, k)



=== paper (cs_nodes_paper_emb.csv) ===
컬럼 수: 833 행 수: 5000
컬럼명: ['paper_id', 'venue_0', 'venue_1', 'venue_2', 'venue_3', 'venue_4', 'venue_5', 'venue_6', 'venue_7', 'venue_8'] ...
paper_id     object
venue_0     float64
venue_1     float64
venue_2     float64
venue_3     float64
venue_4     float64
venue_5     float64
venue_6     float64
venue_7     float64
venue_8     float64
dtype: object
샘플 rows:
                           paper_id   venue_0   venue_1   venue_2   venue_3  \
0  https://openalex.org/W3010906965 -0.146592 -0.078469 -0.076921  0.009507   
1  https://openalex.org/W4213446860  0.130415  0.136322 -0.114638 -0.069871   
2  https://openalex.org/W2987460522  0.090419  0.206654 -0.113040  0.113296   
3  https://openalex.org/W3133928066 -0.098734  0.012867 -0.092038  0.129437   
4  https://openalex.org/W4382464460  0.178455  0.029983 -0.099174 -0.114309   

    venue_4   venue_5   venue_6   venue_7   venue_8  ...   emb_758   emb_759  \
0  0.059249 -0.073674 -0.228234  0.123331

In [20]:
# paper feats
paper_df = pd.read_csv("cs_nodes_paper_emb.csv")
paper_df = paper_df.sort_values("paper_id").reset_index(drop=True)  # edge와 같은 순서로 맞추기
paper_feats = torch.tensor(paper_df.drop(columns=["paper_id"]).values, dtype=torch.float32)

print("paper_df.index range:", paper_df.index.min(), paper_df.index.max(), "행 수:", len(paper_df))


paper_df.index range: 0 4999 행 수: 5000


In [35]:
import pandas as pd

# 파일 경로
author_path = "cs_nodes_author_emb.csv"
concept_path = "nodes_concepts_with_emb.csv"

# Author 확인
author_df = pd.read_csv(author_path)
print("\n=== Author ===")
print("index range:", author_df.index.min(), author_df.index.max(), "행 수:", len(author_df))
print("author_id 샘플:", author_df["author_id"].head().tolist())
print("author_id 마지막 샘플:", author_df["author_id"].tail().tolist())

# Concept 확인
concept_df = pd.read_csv(concept_path, encoding="latin1")
print("\n=== Concept ===")
print("index range:", concept_df.index.min(), concept_df.index.max(), "행 수:", len(concept_df))
print("concept_id 샘플:", concept_df["concept_id"].head().tolist())
print("concept_id 마지막 샘플:", concept_df["concept_id"].tail().tolist())



=== Author ===
index range: 0 33606 행 수: 33607
author_id 샘플: [5026718416, 5058947613, 5004260600, 5042863303, 5016535318]
author_id 마지막 샘플: [5029501164, 5048947116, 5047852894, 5078252913, 5101762391]

=== Concept ===
index range: 0 6611 행 수: 6612
concept_id 샘플: ['https://openalex.org/C2778029271', 'https://openalex.org/C177562468', 'https://openalex.org/C63479239', 'https://openalex.org/C55166926', 'https://openalex.org/C4679612']
concept_id 마지막 샘플: ['https://openalex.org/C2776459890', 'https://openalex.org/C87227347', 'https://openalex.org/C140417398', 'https://openalex.org/C122280245', 'https://openalex.org/C134517425']


### 확인 끝 mydata.pkl 추출 시작

Import & 파일 로드

In [39]:
import os
import pandas as pd
import numpy as np
import torch
from scipy.sparse import csr_matrix

# Edge CSV 불러오기
df_ap = pd.read_csv("edge_wrote.csv")        # author–paper
df_pc = pd.read_csv("edge_has_topic.csv")    # paper–concept
df_pp = pd.read_csv("edge_cites.csv")        # paper–paper
df_aa = pd.read_csv("edge_worked_with.csv")  # author–author

# Node CSV 불러오기
paper_df = pd.read_csv("cs_nodes_paper_emb.csv")     # 5000 × 832
author_df = pd.read_csv("cs_nodes_author_emb.csv")   # 33607 × 768
concept_df = pd.read_csv(concept_path, encoding="latin1") # 6612 x 768


노드 개수 확인

In [56]:
na = len(author_df)   # 33607
npapers = len(paper_df)  # 5000
nc = len(concept_df)  # 6612
print(f"#authors={na}, #papers={npapers}, #concepts={nc}")


#authors=33607, #papers=5000, #concepts=6612


Feature Tensor 만들기 (feats)

In [59]:
# paper_id / author_id / concept_id 같은 메타정보 컬럼 제거 후 임베딩만 tensor 변환
paper_feats = torch.tensor(paper_df.drop(columns=["paper_id"]).values, dtype=torch.float32)
author_feats = torch.tensor(author_df.drop(columns=["author_id"]).values, dtype=torch.float32)
concept_feats = torch.tensor(concept_df.drop(columns=["concept_id", "concept_name", "level"]).values,
                             dtype=torch.float32)

print("paper feats:", paper_feats.shape)
print("author feats:", author_feats.shape)
print("concept feats:", concept_feats.shape)

paper feats: torch.Size([5000, 832])
author feats: torch.Size([33607, 768])
concept feats: torch.Size([6612, 768])


Adjancency 만들기

In [64]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

def safe_build_csr(src, dst, nrow, ncol, name="matrix"):
    """
    ✅ 범위 자동 체크 + 초과 인덱스 제거 후 csr_matrix 생성
    src: row index list
    dst: col index list
    nrow: matrix rows
    ncol: matrix cols
    name: 출력용 matrix 이름
    """
    src = np.array(src)
    dst = np.array(dst)

    # ✅ 범위 벗어난 값 자동 필터링
    mask = (src >= 0) & (src < nrow) & (dst >= 0) & (dst < ncol)
    dropped = len(src) - mask.sum()

    if dropped > 0:
        print(f"[⚠️] {name}: {dropped} edges dropped (out-of-bound indices)")

    src = src[mask]
    dst = dst[mask]

    # ✅ CSR 생성
    data = np.ones(len(src), dtype=np.int8)
    mat = csr_matrix((data, (src, dst)), shape=(nrow, ncol))

    # ✅ 결과 확인
    print(f"[✅] {name} built with shape {mat.shape}, edges: {mat.nnz}")
    return mat

# 📌 feat shape 기반 노드 수 (반드시 feature shape에서 가져오기)
npapers = paper_feats.shape[0]      # 5000
na = author_feats.shape[0]          # 33607
nc = concept_feats.shape[0]         # 6612

# ✅ 각 adjacency 생성 (자동 검증 + 안전 생성)
A_AP = safe_build_csr(df_ap["src"], df_ap["dst"], na, npapers, name="A_AP")
A_PA = A_AP.T

A_PC = safe_build_csr(df_pc["src"], df_pc["dst"], npapers, nc, name="A_PC")
A_CP = A_PC.T

A_PP = safe_build_csr(df_pp["src"], df_pp["dst"], npapers, npapers, name="A_PP")
A_AA = safe_build_csr(df_aa["src"], df_aa["dst"], na, na, name="A_AA")


[✅] A_AP built with shape (33607, 5000), edges: 40591
[⚠️] A_PC: 2871 edges dropped (out-of-bound indices)
[✅] A_PC built with shape (5000, 6612), edges: 79367
[✅] A_PP built with shape (5000, 5000), edges: 8186
[✅] A_AA built with shape (33607, 33607), edges: 984700


MPS 만들기

In [66]:
# PAP = A_AP * A_PA
M_PAP = (A_AP @ A_PA).astype(np.int8)
M_PAP.setdiag(0); M_PAP.eliminate_zeros()

# PCP = A_PC * A_CP
M_PCP = (A_PC @ A_CP).astype(np.int8)
M_PCP.setdiag(0); M_PCP.eliminate_zeros()

def scipy_to_torch_sparse(M):
    coo = M.tocoo()
    idx = torch.from_numpy(np.vstack([coo.row, coo.col])).long()
    vals = torch.from_numpy(coo.data.astype(np.float32))
    return torch.sparse_coo_tensor(idx, vals, torch.Size(coo.shape)).coalesce()

mps = {
    "PAP": scipy_to_torch_sparse(M_PAP),
    "PCP": scipy_to_torch_sparse(M_PCP),
}

for name, mat in mps.items():
    print(f"{name} shape={mat.shape}, nnz={mat._nnz()}")


PAP shape=torch.Size([33607, 33607]), nnz=984548
PCP shape=torch.Size([5000, 5000]), nnz=24995000


nei_index 만들기

In [68]:
def build_nei_index(M, max_neighbors=20, seed=42):
    rng = np.random.default_rng(seed)
    neighbors = []
    for i in range(M.shape[0]):
        idx = M[i].indices
        if len(idx) > max_neighbors:
            idx = rng.choice(idx, size=max_neighbors, replace=False)
        neighbors.append(idx)
    return neighbors

nei_index = {
    "PAP": build_nei_index(M_PAP, max_neighbors=20, seed=42),
    "PCP": build_nei_index(M_PCP, max_neighbors=20, seed=42),
}

# 확인
print("PAP node0 neighbors:", nei_index["PAP"][0])
print("PCP node0 neighbors:", nei_index["PCP"][0])


PAP node0 neighbors: [27762 23183 13559 12450 10178  5994  4428]
PCP node0 neighbors: [   5 1774 2876 2423 3162  305 3645 4056 4160 3922  482 4086  279 2664
 3988 1163 4655  418 2603 3482]


pos 만들기

In [70]:
pos = {
    "ap": torch.tensor([df_ap["src"].to_numpy(), df_ap["dst"].to_numpy()], dtype=torch.long),
    "pc": torch.tensor([df_pc["src"].to_numpy(), df_pc["dst"].to_numpy()], dtype=torch.long),
    "pp": torch.tensor([df_pp["src"].to_numpy(), df_pp["dst"].to_numpy()], dtype=torch.long),
    "aa": torch.tensor([df_aa["src"].to_numpy(), df_aa["dst"].to_numpy()], dtype=torch.long),
}

for k, v in pos.items():
    print(k, v.shape)


ap torch.Size([2, 40758])
pc torch.Size([2, 82238])
pp torch.Size([2, 8186])
aa torch.Size([2, 984700])


/var/folders/yv/3ng84z553ljcmc8pmfpbj_540000gn/T/ipykernel_9418/3676224803.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  "ap": torch.tensor([df_ap["src"].to_numpy(), df_ap["dst"].to_numpy()], dtype=torch.long),


최종 저장

In [72]:
save_dict = {
    "feats": {
        "paper": paper_feats,
        "author": author_feats,
        "concept": concept_feats
    },
    "mps": mps,
    "nei_index": nei_index,
    "pos": pos,
}

os.makedirs("data", exist_ok=True)
torch.save(save_dict, "data/mydata.pkl")
print("Saved: data/mydata.pkl")


Saved: data/mydata.pkl


### 확인

In [74]:
import torch

# 저장된 파일 불러오기
data = torch.load("data/mydata.pkl")

print("=== Keys in mydata.pkl ===")
for k in data.keys():
    print("-", k)

# feats 확인
print("\n=== Node Feature Tensors (in data['feats']) ===")
if "feats" in data:
    for node_type in ["paper", "author", "concept"]:
        if node_type in data["feats"] and data["feats"][node_type] is not None:
            feat = data["feats"][node_type]
            print(f"{node_type}: shape={feat.shape}, dtype={feat.dtype}")
        else:
            print(f"{node_type}: ❌ 없음")
else:
    print("❌ 'feats' key가 존재하지 않습니다. HGMAE 호환 구조가 아닙니다.")
    
# mps 확인
print("\n=== mps ===")
for name, mat in data["mps"].items():
    print(f"{name}: shape={mat.shape}, nnz={mat._nnz()}")

# nei_index 확인
print("\n=== nei_index ===")
for name, idx_list in data["nei_index"].items():
    print(f"{name}: num_nodes={len(idx_list)}, 첫 노드 neighbors={idx_list[0]}")

# pos 확인
print("\n=== pos ===")
for name, edge_index in data["pos"].items():
    print(f"{name}: shape={edge_index.shape}, 예시={edge_index[:, :5]}")


=== Keys in mydata.pkl ===
- feats
- mps
- nei_index
- pos

=== Node Feature Tensors (in data['feats']) ===
paper: shape=torch.Size([5000, 832]), dtype=torch.float32
author: shape=torch.Size([33607, 768]), dtype=torch.float32
concept: shape=torch.Size([6612, 768]), dtype=torch.float32

=== mps ===
PAP: shape=torch.Size([33607, 33607]), nnz=984548
PCP: shape=torch.Size([5000, 5000]), nnz=24995000

=== nei_index ===
PAP: num_nodes=33607, 첫 노드 neighbors=[27762 23183 13559 12450 10178  5994  4428]
PCP: num_nodes=5000, 첫 노드 neighbors=[   5 1774 2876 2423 3162  305 3645 4056 4160 3922  482 4086  279 2664
 3988 1163 4655  418 2603 3482]

=== pos ===
ap: shape=torch.Size([2, 40758]), 예시=tensor([[ 8136, 17961,  1297, 13091,  5067],
        [    0,     0,     0,     0,     1]])
pc: shape=torch.Size([2, 82238]), 예시=tensor([[   0,    0,    0,    0,    0],
        [3526, 1937,  552, 5006,  539]])
pp: shape=torch.Size([2, 8186]), 예시=tensor([[   0,    0,    1,    2,    2],
        [4517, 3007, 2972, 

In [76]:
ls -lh data/mydata.pkl

-rw-r--r--@ 1 seoyeonko  staff   651M Sep 27 16:32 data/mydata.pkl
